## Construct a SOFR curve and value an IRS

This notebook illustrates how rateslib can be used to calibrate a SOFR curve against input instruments, then used to price and risk an interest rate swap.

All code taken from https://rateslib.readthedocs.io/en/1.1.x/z_swpm.html


In [ ]:
import platform
import warnings

if platform.system() == 'Emscripten':
    # Install rateslib if running on jupyterlite.
    import micropip
    await micropip.install('rateslib')
    package_list = micropip.list()
    print(f"Loaded version {package_list["rateslib"].version} of rateslib.")

with warnings.catch_warnings(action="ignore"):
    from datetime import datetime as dt
    import pandas as pd
    import rateslib as rl
    

### Curve data

First set-up the market data inputs.

In [ ]:
from datetime import datetime as dt
import pandas as pd
import rateslib as rl

as_of_date = dt(2023, 8, 17)
effective_date = rl.add_tenor(as_of_date, "2b", "F", "nyc")

data = pd.DataFrame({
    "Term": ["1W", "2W", "3W", "1M", "2M", "3M", "4M", "5M", "6M", "7M", "8M", "9M", "10M", "11M", "12M", "18M", "2Y", "3Y", "4Y"],
    "Rate": [5.30111, 5.30424, 5.30657, 5.31100, 5.34800, 5.38025, 5.40915, 5.43078, 5.44235, 5.44950, 5.44878, 5.44100, 5.42730, 5.40747, 5.3839, 5.09195, 4.85785, 4.51845, 4.31705],
})
data["Termination"] = [rl.add_tenor(effective_date, tenor, "F", "nyc") for tenor in data["Term"]]
data


A visual representation of the inputs.

In [ ]:
pd.Series(data.Rate.values, data.Termination).plot(style='+-');


### Curve structure

Create a curve with node dates matching the inputs, and give the interpolation information.

In [ ]:
sofr_curve = rl.Curve(
    id="sofr_curve",
    convention="Act360",
    calendar="nyc",
    modifier="MF",
    interpolation="log_linear",
    nodes={
        **{as_of_date: 1.0},  # <- this is today's DF,
        **{term_dt.to_pydatetime(): 1.0 for term_dt in data["Termination"]},
    }
)


### Create a Solver

Calibrate the curve to the input instruments using a Solver.

In [ ]:
instruments = [
    rl.IRS(termination=term.to_pydatetime(), 
        effective=effective_date,
        spec="usd_irs",
        curves="sofr_curve"
       ) for term in data["Termination"]
]

solver = rl.Solver(
    curves=[sofr_curve],
    instruments=instruments,
    s=data["Rate"],
    instrument_labels=data["Term"],
    id="us_rates",
)

### Curve results

Can now read the discount factors from the calibrated curve.

In [ ]:
data["DF"] = [float(sofr_curve[term]) for term in data["Termination"]]
data

### Pricing an Instrument

Create an Interest Rate Swap to price.

In [ ]:
irs = rl.IRS(
    effective=dt(2023, 11, 21),
    termination=dt(2025, 2, 21),
    notional=-100e6,
    fixed_rate=5.40,
    curves="sofr_curve",
    spec="usd_irs",
)


Then use the solver to price it.

In [ ]:
irs.npv(solver=solver).real


### Delta

The solver can be used to get delta senstivites,

In [ ]:
delta = irs.delta(solver=solver)
delta

Show the strip delta graphically.

In [ ]:
ccy_delta = delta["usd"]["usd"]
pd.Series(ccy_delta.values,
          ccy_delta.index.get_level_values("label")).plot.bar();

In [ ]:
float(ccy_delta.sum())


In [ ]:
irs.analytic_delta(curve=sofr_curve).real


### Gamma

The solver can also be used to get cross-gamma sensitivities.

In [ ]:
irs.gamma(solver=solver).sum().sum()


### Cashflows

Show a summary of cashflows to be paid, or get a more detailed breakdown of cashflow information.

In [ ]:
irs.cashflows_table(solver=solver)


In [ ]:
irs.cashflows(solver=solver)
